## Bracket Challenge Predictor

Reading data

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

from xgboost import XGBRegressor

In [80]:
team_matchups = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Tournament Matchups.csv")

away_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Away.csv")
homme_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Home.csv")
neutral_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Barttorvik Neutral.csv")
conf_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Conference Stats.csv")
team_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Team Results.csv")
team_v_ranked = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/TeamRankings.csv")
team_shot_dis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Shooting Splits.csv")

ken_def = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Defense.csv")
ken_off = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Offense.csv")
ken_eff = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Efficiency.csv")
ken_height = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Height.csv")
ken_mis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Miscellaneous Team Stats.csv")
ken_pt_dis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Point Distribution.csv")
ken_summ = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/INT _ KenPom _ Summary.csv")
ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")

conf_mapping = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ NCAAM Conference and ESPN Team Name Mapping.csv")

cur_coaches = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/REF _ Current NCAAM Coaches.csv")
coach_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Coach Results.csv")

seed_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Seed Results.csv")
upset_count = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Count.csv")
upset_inf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/Upset Seed Info.csv")

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_28025/197528209.py:18: DtypeWarning: Columns (144) have mixed types. Specify dtype option on import or set low_memory=False.
  ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/data/DEV _ March Madness.csv")


since ken_ datasets don't have TEAMNO var will first create seperate lookup table to create a column for its respective id

In [81]:
team_ids = team_matchups[['YEAR','TEAM','TEAM NO']].drop_duplicates()

ken_def.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_off.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_eff.rename(columns={'Season':'YEAR','Team':'TEAM'},inplace=True)
ken_height.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_mis.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_pt_dis.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)
ken_summ.rename(columns={'Season':'YEAR','TeamName':'TEAM'},inplace=True)


ken_def = ken_def.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_off = ken_off.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_eff = ken_eff.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_height = ken_height.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_mist = ken_mis.merge(team_ids,on=['YEAR','TEAM'],how='left')
ken_pt_dis = ken_pt_dis.merge(team_ids,on=['YEAR','TEAM'],how='left')


In [82]:
conf_ids = conf_perf[['YEAR','CONF','CONF ID']].drop_duplicates()

ken_conf.rename(columns={'Season':'YEAR','Short Conference Name':'CONF'},inplace=True)

ken_conf = ken_conf.merge(conf_ids,on=['YEAR','CONF'],how='left')

In [86]:
away_stats.columns

Index(['YEAR', 'TEAM NO', 'TEAM ID', 'TEAM', 'SEED', 'ROUND', 'BADJ EM',
       'BADJ O', 'BADJ D', 'BARTHAG', 'GAMES', 'W', 'L', 'WIN%', 'EFG%',
       'EFG%D', 'FTR', 'FTRD', 'TOV%', 'TOV%D', 'OREB%', 'DREB%', 'OP OREB%',
       'OP DREB%', 'RAW T', '2PT%', '2PT%D', '3PT%', '3PT%D', 'BLK%', 'BLKED%',
       'AST%', 'OP AST%', '2PTR', '3PTR', '2PTRD', '3PTRD', 'BADJ T',
       'AVG HGT', 'EFF HGT', 'EXP', 'TALENT', 'FT%', 'OP FT%', 'PPPO', 'PPPD',
       'ELITE SOS', 'WAB', 'BADJ EM RANK', 'BADJ O RANK', 'BADJ D RANK',
       'BARTHAG RANK', 'EFG% RANK', 'EFGD% RANK', 'FTR RANK', 'FTRD RANK',
       'TOV% RANK', 'TOV%D RANK', 'OREB% RANK', 'DREB% RANK', 'OP OREB% RANK',
       'OP DREB% RANK', 'RAW T RANK', '2PT% RANK', '2PT%D RANK', '3PT% RANK',
       '3PT%D RANK', 'BLK% RANK', 'BLKED% RANK', 'AST% RANK', 'OP AST% RANK',
       '2PTR RANK', '3PTR RANK', '2PTRD RANK', '3PTRD RANK', 'BADJT RANK',
       'AVG HGT RANK', 'EFF HGT RANK', 'EXP RANK', 'TALENT RANK', 'FT% RANK',
       'OP 

In [ ]:
away_stats.loc[['YEAR','TEAM NO','BADJ EM','BADJ O','BADJ D','WIN%','EFG%','EFG%D']]